# Utility to create stable file used for reproduciblity checks

## v2.2 Code

In [ ]:
import pickle
import numpy as np
import os.path as osp
import os
import pandas as pd
import tensorflow as tf
import sys
sys.path.append('..')
from moisture_rnn_pkl import pkl2train
from moisture_rnn import RNNParams
from data_funcs import build_train_dict, combine_nested
from utils import read_yml, read_pkl, print_dict_summary, retrieve_url

In [ ]:
filename = "fmda_rocky_202403-05_f05.pkl"
retrieve_url(
    url = f"https://demo.openwfm.org/web/data/fmda/dicts/{filename}", 
    dest_path = f"../data/{filename}")

In [ ]:
pkl_file = f"../data/{filename}"
case_name = "CPTC2_202403_set_2"
# Destination File
outfile = "../data/reproducibility_dict_v2.2.pkl"

## Read Data and Extract Case

In [ ]:
params_data = read_yml("../params_data.yaml")
params_data

In [ ]:
train = build_train_dict([pkl_file], params_data=params_data,
                         spatial=False, forecast_step=1)

In [ ]:
train.keys()

In [ ]:
single_case = train[case_name].copy()
print_dict_summary(single_case)

In [ ]:
spatial_case = combine_nested(train.copy())
print_dict_summary(spatial_case)

In [ ]:
# rnn_dat_single = RNNData(single_case, params)
# rnn_dat_spatial = RNNData(single_case, params)

## Add Reproducibility Info

In [ ]:
params = read_yml('../params.yaml', subkey="rnn_repro_NEW")
params

In [ ]:
params_single = params.copy()
# Remove keys not used for single case
keys_to_remove = ['features_list_spatial', 'space_fracs']
for key in keys_to_remove:
    params_single.pop(key)
params_single['features_list'] = params_single.pop('features_list_single')

# Remove keys not used for spatial case
params_spatial = params.copy()
params_spatial.pop('features_list_single')
params_spatial['features_list'] = params_spatial.pop('features_list_spatial')

In [ ]:
repro_info_single = {
    'phys_initialize': {
        'fitted_weights_hash': '2ac4a00bf08f8e21d9661daecbbd7fdc',
        'preds_hash': 'fe059c66e8ea8fe8b0df5ddf817f16cb'
    },
    'rand_initialize':{
        'fitted_weights_hash': '8b1916fce4461ebb7f345b4810d97c6d',
        'preds_hash': 'bb3915900bf8d8c3420cc2147678c447'
    },
    'env_info':{
        'py_version': sys.version[0:6],
        'tf_version': tf.__version__,
        'seed': 123
    },
    'params': RNNParams(params_single)
}

single_case['repro_info'] = repro_info_single

In [ ]:
repro_info_spatial = {
    'phys_initialize': "Not Implemented Yet",
    'rand_initialize':{
        'fitted_weights_hash': '77d0580aace582c1a622a757827ff8d4',
        'preds_hash': '18df0540385f3cca6840f035c90cf3cd'
    },
    'env_info':{
        'py_version': sys.version[0:6],
        'tf_version': tf.__version__,
        'seed': 123
    },
    'params': RNNParams(params_spatial)
}
spatial_case['repro_info'] = repro_info_spatial

## Write Output

In [ ]:
with open(outfile, 'wb') as file:
    print(f"Writing file: {outfile}")
    pickle.dump({'single': single_case, 'spatial': spatial_case}, file)